In [1]:
from fh import FH 
import fh
from treeBuilder import TreeBuilder as TB
import copy
import pickle
import arpeggiator
from spatializer import Spatializer
from LFOFM import *
import PydalChanel as pydalmodule
from OSC import *

wp = WavePlayer()
lp = FH()

def sendMsgSC(addr, *args):
    msg = OSCMessage()
    msg.setAddress(addr)
    msg.append(args)
    wp.superColliderClient.send(msg)
    

plotWave = wp.plotWave
oneHitShift = fh.oneHitShift
read = pydalmodule.read
rootScale = lp.rootScale

arpPydal = pydalmodule.getInstance(('127.0.0.1', 34567))

In [2]:
#IGNORE THIS for Max backend piece
pydal = pydalmodule.getInstance()

end = pydal.end
tempo = pydal.setTempo
stripSymbols = lambda beats : map(lambda beat : list(beat[1])[0], beats)

ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)
ch3 = pydal.newChannel(3)
ch4 = pydal.newChannel(4)
ch5 = pydal.newChannel(5)
ch5 = pydal.newChannel(5)

In [3]:
#sustainPad -> arpeggiator setup

s = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp, 0)
s.spatialize = False
s.separateChannels = False
s.broadcasting = False
lp.addForwardingHandler(0, s)
# s.loadChordsFromFile("songV4.chords")

a = arpeggiator.Arpeggiator(0, arpPydal, read("0 1 2", 1, "arp"))
a.normalForwardingBehavior = False
lp.addForwardingHandler(0, a)

s2 = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp, 1)
s2.spatialize = False
s2.separateChannels = False
s2.broadcasting = False
lp.addForwardingHandler(1, s2)

b = arpeggiator.Arpeggiator(1, arpPydal, read("0 1 2", 1, "arp"))
b.normalForwardingBehavior = False
lp.addForwardingHandler(1, b)



def play(patStr, dur, arp):
    pat = read(patStr, dur, "arp")
    arp.pattern = pat
    arp.channel.pydalPattern = pat

## ALGO-FOLK SETUP

In [ ]:
lp.resetButtonDestinations([0]*20 + ["-1"]*60)
s.loadChordsFromFile("algofolk/4.chords")
s.broadcasting = True
s2.broadcasting = True
tempo(120)

In [ ]:
#TODO - be able to change chords without restarting pattern 
#make this selectable per-chord as a symbol extension on pydal-chords
#also add this as a toggle on the chord-pad 
#TODO - add velocity controls 
#   - fwd sustain-velocity to chords/arp
#   - add velocity annotation in arp
#   - make velocity controllable via LFOFM
#TODO - note length - add note-level annotation in arp, also LFOFM

#tempo(120)

a0 = ("0 1 3 0 2 4", 2)
a1 = ("<0, 2 1> <1, 1*2, 3 2 1> [4 [(1 2, 3 2, 0)] <4*2, 3*1>]", 2)
a2 = ("<[0 1 2 3], [0 2 3 4]>", 1)
a3 = ("<<0 1, 1 0> <1 2, 2 1>, <2 3, 3 2> <3 4, 4 3>, 0 1 2 3>", 1)
a4 = ("0 ~ 2 0 1 2 <4, 3> 2 1 <4, 2> 2 1", 3)
a5 = ("[0 0b 0 1] [1 1 2 2] [2 3 3 3]", 3)
a6 = ("[0 ~ 1 2 ~ 3, 2 ~ 3 4 ~ 0]", 1)
a7 = ("<[<0, 0, 0 1 2 3>, 2, 4], [0 2, 1, <3, 4 [4 3]>]>", 1)

c1 = read("da [<df, dg> <dc, dd>] ed [ec <dc, ef>]", 8, "chord")
c2 = read("ea [<ff, eg> <ec, ed>] fd [fc <ec, ff>]", 8, "chord")
c3 = read("ba [<bf, bg> <bc, bd>] bd [bc <bc, bf>]", 8, "chord")
c4 = read("fa fb fc fd fe ff fg fh", 8, "chord")


arp = a1
chrd = c2

In [ ]:
#tempo(160)

s.swappingChords = True
a3 = ("<<0 1, 1 <0, ~>> <1 2, 2 <1, ~>>, <2 3, 3 2> <3 4, 4 3>, 0 1 2 3>", 1)
a4 = ("0 ~ 2 0 1 2 <4, 3> 2 1 <4, 2> 2 1", 4)
a5 = ("0 0 0 1 1 1 2 2 2 3 3 3", 4)

arp = a7
chrd = c4

In [ ]:
tempo(120)
play(arp[0], arp[1], a)
ch1.play(chrd, [0])

In [ ]:
s.debug = False
a.debug=True

In [ ]:
a.sendNoteDuration(1)

## DRONE ADDITIONAL SETUP

In [ ]:
#load for drone song
#set divider in chordpad to 20
# masterclock, spatializer, lfofm, faders, chordpad, arpeggiator
# shift pad lower by 1 row

lp.resetButtonDestinations([0]*20 + ["-2"]*60)
s2.loadChordsFromFile("droneSong3.chords")
s2.broadcasting = False
s2.separateChannels = True

spray = 12
freq = 13
pitch = 14
rpitch = 15
fdbk = 16
mix = 17
params = [spray, freq, pitch, rpitch, fdbk, mix]

def stopAllWaves():
    for p in params:
        wp.stopWave(p)

def resetWaves():
    vals = [-64, 64, 64, -64, -64, 64]
    for i in range(len(params)):
        wp.startWave(params[i], vals[i])
    stopAllWaves()

# 8 "lines"
# l,2,3 - rows 1,2,3 - 1 beat per chord, 2 beats on last
# 4 - row 3 - 1248537 - 1 beat per chord, 2 beats on last
# 5,6 - row 4 - 12545456 then 1-8
# 7 - row 1 - 123, row 3 - 4567 1 beat per chord, 2 beats on last
# 8 - row 3 - 1234, row 6 - 567 1 beat per chord, 2 beats on last

In [ ]:
wp.startWave(freq, Saw(freq=0.1, phase=Sin(freq=0.2))*40+60)
wp.startWave(fdbk, Sin(freq=0.2)*50)
wp.startWave(spray, Tri(freq=.1,phase=Sin(freq=0.2))*50)
# wp.startWave(17, 50)

In [ ]:
plotWave(Tri(freq=0.4, phase=Sin(phase=Saw(0.1))*0.1), 0, 40)

In [ ]:
wp.startWave(spray, Tri(freq=0.3, phase=Sin()*0.3))
wp.startWave(freq, 60)
wp.startWave(pitch, Sin(0.1, phase=Tri())*40-20)
wp.startWave(rpitch, Sin(0.3)*30-30)
wp.startWave(fdbk, 64)
wp.startWave(mix, Saw(0.2, phase=Sin())*12 - 32)

In [ ]:
wp.startWave(spray, -64)
wp.startWave(freq, 64)
wp.startWave(pitch, 64)
wp.startWave(rpitch, -64)
wp.startWave(fdbk, -64)
wp.startWave(mix, 64)

In [ ]:
w1 = Sin(0.1, phase=Tri())*40-20

In [ ]:
resetWaves()

## RHYTHM WARP SETUP

In [4]:
tempo(100)
lp.resetButtonDestinations([0]*20 + ["-0"]*30 + ["-1"]*30)
lp.loadScenesFromFile("uberstitious/4.scenes")
s.loadChordsFromFile("uberstitious/organ.chords")

In [5]:
ch1.play(pydalmodule.LoopPattern(lp.loops[1][0]), [1])

In [ ]:
type({5})

In [ ]:
ch1.play(read("bd"))

In [ ]:
#score
# 120 122 phase
# sync
# steady - wave 
# steady - wave increase wave amp
# sync
# wave - wave same wave
# wave - wave increase amp
# waves diverge
# make waves same again
# sync w/o stopping waves

In [ ]:
sendMsgSC("/changeChannelTempo", 122, 2)
sendMsgSC("/changeChannelTempo", 120, 1)

In [ ]:
sendMsgSC("/syncClocks", "both", "master", 1, 2)

In [6]:
ch1.stop()
ch2.stop()

In [ ]:
wp.startWaveOSC("chan1", "/changeChannelTempo", Tri(freq=0.5)*50 + 110, "sc", 1)
wp.startWaveOSC("chan2", "/changeChannelTempo", Tri(freq=0.5)*50 + 110, "sc", 2)

In [ ]:
wp.stopWave("chan1")
wp.stopWave("chan2")

## FUNK SETUP

In [ ]:
#load for funky jam
#set divider in chordpad to 50
#make sure stems are "0-" and "-1"

# masterClock, multiLoop, chordPad, spatialize 

#todo - a different groove for a proper "b" section (currently the grooves are kind of the same)

s2.loadChordsFromFile("phillyjam/horns2.chords")
s.loadChordsFromFile("phillyjam/organ3.chords")
lp.loadScenesFromFile("phillyjam/philly2.scenes")

s.broadcasting = False
s2.broadcasting = False

In [ ]:
s.broadcasting = False
s2.broadcasting = False

In [ ]:
play("(0,1,2)*6", 1, a)
play("(0,1,2)*6", 1, b)
a.sendNoteDuration(0.1)
b.sendNoteDuration(0.1)

## sampler DnB setup

In [ ]:
s3 = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp, 2)
s3.spatialize = False
s3.separateChannels = False
s3.broadcasting = False
lp.addForwardingHandler(2, s3)

c = arpeggiator.Arpeggiator(2, arpPydal, read("0 1 2", 1, "arp"))
c.normalForwardingBehavior = False
lp.addForwardingHandler(2, c)

#set divider in chordpad to 50, and change 20-divider to have stem "-2"
# allinone, maxLink, multiloop, chordpad, spatialize, arpeggiator
# shift pad lower by 1 row

In [ ]:
s3.broadcasting = False
s3.normalForwardingBehavior = False
c.normalForwardingBehavior = True

In [ ]:
c.sendNoteDuration(0.025)

In [ ]:
a1 = "0*4"
a2 = "3 0 4 0 1 0 2 0 "
play(a1, 1, c)

In [ ]:
s3.saveChordsToFile("phillyjam/dnb.chords")

In [ ]:
chanInd = 2
bankNum = 1
lp.loops[chanInd][bankNum] = goodBass
lp.loopInfo[chanInd][bankNum]["button"] = 20
lp.loopInfo[chanInd][bankNum]["playing"] = False
lp.roots[chanInd] = lp.roots[2]
lp.scales[chanInd] = lp.scales[2]

# SCRATCH

# SCRATCH

# SCRATCH

In [ ]:
# s2.saveChordsToFile("phillyjam/horns4.chords")
# s.saveChordsToFile("phillyjam/organ4.chords")
# lp.saveScenesToFile("phillyjam/philly3.scenes")

In [ ]:
wp.plotWave(Sin(freq=0.1, phase=Sin(freq=0.2))*40+60)

In [ ]:
a.sendNoteDuration(0.1)

In [ ]:
play("(0, 1, 2)*4", 1)

In [ ]:
p1 = read("da [ab ac]", 16, "chord")

In [ ]:
p1.render()

In [ ]:
ch1.play(p1, [1])

In [ ]:
s.broadcasting = True
s2.broadcasting = True

In [ ]:
ch1.stop()

In [ ]:
lp.loops[2]

In [ ]:
s = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp)
s.spatialize = False
s.separateChannels = True
lp.addForwardingHandler(1, s)
s.loadChordsFromFile("songV4.chords")
def toggleSustain():
    s.sustaining = not s.sustaining
def toggleSeparateChannels():
    s.separateChannels = not s.separateChannels
lp.topRowFunctions[0] = toggleSustain
lp.topRowFunctions[1] = toggleSeparateChannels


In [ ]:
# ch1.play(read("[a:0_0.25*2, b:1_1.5*3] ", 1, "max"))
ch1.stop()

In [ ]:
rootScale(0, 57, "minor")

In [ ]:
def toggleSustain():
    s.sustaining = not s.sustaining
def toggleSeparateChannels():
    s.separateChannels = not s.separateChannels
lp.topRowFunctions[0] = toggleSustain
lp.topRowFunctions[1] = toggleSeparateChannels

In [ ]:
s.heldNotesOff(1)

In [ ]:
lp.startChannel(1, lambda buf: buf, lp.loops[2][0])

In [ ]:
lp.stopChannel(1)

In [ ]:
a = read("{bd (lt*2, lt, lt), hc <~, sn hc> ~ hc:2}")
b = read("{(bd ~, sn:3 <~, cp>)}")
c = read("((bd:2*2, [bd:3 bd:3] bd), [sn:5 [bd sn:3]], [[bd sn:3] bd], <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>)")
c2 = read("(bd:2*2, [bd:3 bd:3] bd) [sn:5 [bd sn:3]] [[bd sn:3] bd] <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>", 4)
d = read("{~ ~, bin ~ bin ~}")
e = read("{[bin <bin:1, bin, bin*2> bin bot, hc <hc, ho>], bd sn:3}")

'''
bd 3
lt
sn 5
cp
hc:1 
ho
bot
bin 
bend:2
1 freebie 
'''

demo = read("bend:2 bin")

f = read("aa:4_4.75*2 aa:5_5.75*3", 2, 'sample')
f2 = read("hc*2 cp*3", 2)
d2 = read("{~ ~ , bd ~ sn:3 ~}", 4)
g = read("{~ ~ , ab:0_2 ~ ab:3_5 ~}", 4, 'sample')

In [ ]:
ch1.play(read("[bd bd bd, hc (hc, ho, hc*2), bot <bot*2, bot> bot bot*2]", 2))

In [ ]:
ch1.stop()

In [ ]:
s.loadChordsFromFile("songV4.chords")

In [ ]:
rootScale(1, lp.roots[1], "maj5")

In [ ]:
lp.startChannel(1, lambda buf: fh.randTranspose(buf, lp.roots[0], lp.scales[0]), lp.loops[0][1])

In [ ]:
lp.stopChannel(1)

In [ ]:
lp.startChannel(1, lambda buf: fh.warp(buf, 3, 1), lp.loops[1][3])